### Evaluating and visualizing gene ontology (GO) terms associated bacterial peptides in degradation experiment

#### Beginning with: stripped bacterial peptides and GO term abundances in spreadsheets

#### Goal: heatmaps of bacterial GO terms over the 4 timepoints

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from functools import reduce

In [ ]:
## First, combining all bacterial peptides fro